#Imports and Data


In [ ]:
!pip install matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch import nn , optim
from torchvision import datasets, transforms, models
import numpy as np
from PIL import Image
import json

import matplotlib.pyplot as plt
from collections import OrderedDict
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import cv2
import torchvision.transforms as transforms
from torch.utils.data import Subset  # Add this import statement

%matplotlib inline

batch = 4

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((512, 512), antialias=True),
    transforms.Grayscale(num_output_channels=1),
    transforms.PILToTensor(),
])

valid_transform = transforms.Compose([
    transforms.Resize((512, 512), antialias=True),
    transforms.Grayscale(num_output_channels=1),
    transforms.PILToTensor(),
])


In [ ]:
cross_d = datasets.ImageFolder(root='/content/drive/MyDrive/PROPOSED_ACNN/PROPOSED_SACNN1/train/cross', transform=train_transform)
dot_d = datasets.ImageFolder(root='/content/drive/MyDrive/PROPOSED_ACNN/PROPOSED_SACNN1/train/dot', transform=train_transform)


In [ ]:
cross_td = datasets.ImageFolder(root='/content/drive/MyDrive/PROPOSED_ACNN/PROPOSED_SACNN1/test/cross', transform=valid_transform)
dot_td = datasets.ImageFolder(root='/content/drive/MyDrive/PROPOSED_ACNN/PROPOSED_SACNN1/test/dot', transform=valid_transform)

In [ ]:
# cross_d = datasets.ImageFolder(root='/content/drive/MyDrive/train/cross', transform=train_transform)
# dot_d = datasets.ImageFolder(root='/content/drive/MyDrive/train/dot', transform=train_transform)

In [ ]:
# cross_td = datasets.ImageFolder(root='/content/drive/MyDrive/test/cross', transform=valid_transform)
# dot_td = datasets.ImageFolder(root='/content/drive/MyDrive/test/dot', transform=valid_transform)

In [ ]:
class cross_dot(Dataset):
    def __init__(self,cross,dot):
        self.cross = cross
        self.dot   = dot

    def __getitem__(self,index):
        x1=self.cross[index]
        x2=self.dot[index]
        return x1,x2

    def __len__(self):
        return len(self.cross)


In [ ]:
test_data = cross_dot(cross_td,dot_td)
train_data = cross_dot(cross_d,dot_d)

In [ ]:
test_loader = DataLoader(test_data,batch_size=batch,shuffle=False,num_workers=2)

from torch.utils.data import random_split

# Assuming train_dataset is your complete training dataset
train_size = len(train_data)
val_size = int(0.1 * train_size)
train_size = train_size - val_size

train_dataset, val_dataset = random_split(train_data, [train_size, val_size])

# Define your DataLoader for training
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True, num_workers=2)

# Define your DataLoader for validation
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False, num_workers=2)


In [ ]:
!pip install optuna


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.5 MB/s eta 0:00:00


#Multi

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MultiHeadAttention(nn.Module):
    def __init__(self, inplanes, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.inplanes = inplanes
        self.num_heads = num_heads
        assert self.inplanes % self.num_heads == 0, "inplanes must be divisible by num_heads"

        self.head_dim = self.inplanes // self.num_heads

        # Define linear transformations for queries, keys, and values for each head
        self.query_heads = nn.ModuleList([nn.Conv2d(self.inplanes, self.head_dim, 1) for _ in range(self.num_heads)])
        self.key_heads = nn.ModuleList([nn.Conv2d(self.inplanes, self.head_dim, 1) for _ in range(self.num_heads)])
        self.value_heads = nn.ModuleList([nn.Conv2d(self.inplanes, self.head_dim, 1) for _ in range(self.num_heads)])

        # Final linear transformation
        self.w = nn.Conv2d(self.inplanes, self.inplanes, 1)

    def forward(self, xs):
        # Apply linear transformations for query, key, value
        queries = [self.query_heads[i](xs) for i in range(self.num_heads)]
        keys = [self.key_heads[i](xs) for i in range(self.num_heads)]
        values = [self.value_heads[i](xs) for i in range(self.num_heads)]

        # Prepare query, key, value tensors
        queries = [query.view(query.size(0), self.head_dim, -1) for query in queries]
        keys = [key.view(key.size(0), self.head_dim, -1) for key in keys]
        values = [value.view(value.size(0), self.head_dim, -1) for value in values]

        # Compute attention scores
        attention_scores = [torch.matmul(queries[i], keys[i].transpose(-2, -1)) / (self.head_dim ** 0.5) for i in range(self.num_heads)]
        attention_scores = [F.softmax(score, dim=-1) for score in attention_scores]

        # Apply attention to values
        head_outputs = [torch.matmul(attention_scores[i], values[i]) for i in range(self.num_heads)]
        head_outputs = [output.view(output.size(0), self.head_dim, xs.size(2), xs.size(3)) for output in head_outputs]

        # Concatenate head outputs along the channel dimension
        multi_head_output = torch.cat(head_outputs, dim=1)

        # Final linear transformation
        multi_head_output = self.w(multi_head_output)

        # Add residual connection
        output = xs + multi_head_output
        return output

class CNNP(nn.Module):
    def __init__(self):
        super(CNNP, self).__init__()
        channel = 32

        self.conv2 = nn.Sequential(
            nn.Conv2d(1, channel, 5, 1, padding=2),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, channel, 3, 1, padding=1),
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(1, channel, 7, 1, padding=3),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, channel, 3, 1, padding=1),
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, channel, 3, 1, padding=1),
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.conv6 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.conv7 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.conv8 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.conv9 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.conv10 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.attn = MultiHeadAttention(channel, num_heads=8)
        self.pool = nn.AvgPool2d(2, 2)
        self.resize = transforms.Resize(512)

    def forward(self, images):
        out2 = self.conv2(images)
        out3 = self.conv3(images)
        pool2 = self.pool(self.pool(self.pool(out2)))
        pool3 = self.pool(self.pool(self.pool(out3)))

        attn2 = self.attn(pool2)
        attn3 = self.attn(pool3)

        attn2 = self.resize(attn2)
        attn3 = self.resize(attn3)

        out_f = out3 + attn3 + out2 + attn2
        out4 = self.conv4(out_f)
        out5 = self.conv5(out4 + out_f)
        out6 = self.conv6(out4 + out_f)
        out7 = self.conv7(out6 + out_f)
        out8 = self.conv8(out7 + out_f)
        out9 = self.conv9(out8 + out_f)
        out10 = self.conv10(out9 + out_f)

        return out10

In [ ]:
import optuna

# Define the objective function for Optuna
def objective(trial):
    # Hyperparameters to tune
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [4, 8, 16, 32])

    # Adjust the DataLoader for the suggested batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Initialize the model, loss function, and optimizer with the suggested hyperparameters
    model = CNNP().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)

    epochs = 50
    best_val_accuracy = 0.0
    patience = 2
    early_stop_counter = 0

    model.train()

    for epoch in range(epochs):
        total_accuracy_cross_to_dot = 0
        total_accuracy_dot_to_cross = 0

        if epoch % 5 == 0:
            torch.save(model.state_dict(), '/content/drive/MyDrive/PROPOSED_ACNN/PROPOSED_SACNN1/model_parameter/model_state_epoch_{epoch + 1}.pth')

        for i, (cross, dot) in enumerate(train_loader):
            optimizer.zero_grad()

            output_cross_to_dot = model(cross[0].float().to(device))
            loss_cross_to_dot = criterion(output_cross_to_dot, dot[0].float().to(device))
            loss_cross_to_dot.backward()
            optimizer.step()

            predictions_cross_to_dot = torch.round(output_cross_to_dot).squeeze().float()
            correct_cross_to_dot = torch.eq(predictions_cross_to_dot, dot[0].float().to(device).squeeze()).sum().item()
            accuracy_cross_to_dot_batch = correct_cross_to_dot / (batch_size * 512 * 512)
            total_accuracy_cross_to_dot += accuracy_cross_to_dot_batch

        average_accuracy_cross_to_dot = total_accuracy_cross_to_dot / len(train_loader)
        print(f'[{epoch + 1}] Cross-to-Dot Average Accuracy: {average_accuracy_cross_to_dot:.4f}')

        model.eval()
        with torch.no_grad():
            total_val_accuracy_cross_to_dot = 0

            for val_cross_img, val_dot_img in val_loader:
                val_output_cross_to_dot = model(val_cross_img[0].float().to(device))
                val_loss_cross_to_dot = criterion(val_output_cross_to_dot, val_dot_img[0].float().to(device))

                val_predictions_cross_to_dot = torch.round(val_output_cross_to_dot).squeeze().float()
                val_correct_cross_to_dot = torch.eq(val_predictions_cross_to_dot, val_dot_img[0].float().to(device).squeeze()).sum().item()
                val_accuracy_cross_to_dot = val_correct_cross_to_dot / (batch_size * 512 * 512)

                total_val_accuracy_cross_to_dot += val_accuracy_cross_to_dot

            average_val_accuracy_cross_to_dot = total_val_accuracy_cross_to_dot / len(val_loader)
            print(f'[{epoch + 1}] Validation Accuracy - Cross-to-Dot: {average_val_accuracy_cross_to_dot:.4f}')

            if average_val_accuracy_cross_to_dot > best_val_accuracy:
                best_val_accuracy = average_val_accuracy_cross_to_dot
                early_stop_counter = 0
            else:
                early_stop_counter += 1

            if early_stop_counter >= patience:
                print("Early stopping! Validation accuracy did not improve for the last {} epochs.".format(patience))
                break

        scheduler.step()

    return best_val_accuracy

# Create a study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best trial
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2024-06-26 17:14:48,399] A new study created in memory with name: no-name-debcfae6-7d32-409a-894f-5e0777d3e541
<ipython-input-17-e3a2d4e3b2ad>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
<ipython-input-17-e3a2d4e3b2ad>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-1)


[1] Cross-to-Dot Average Accuracy: 0.0460
[1] Validation Accuracy - Cross-to-Dot: 0.1354
[2] Cross-to-Dot Average Accuracy: 0.1431
[2] Validation Accuracy - Cross-to-Dot: 0.1308
[3] Cross-to-Dot Average Accuracy: 0.1611
[3] Validation Accuracy - Cross-to-Dot: 0.1600
[4] Cross-to-Dot Average Accuracy: 0.1733
[4] Validation Accuracy - Cross-to-Dot: 0.1618
[5] Cross-to-Dot Average Accuracy: 0.1743
[5] Validation Accuracy - Cross-to-Dot: 0.1619
[6] Cross-to-Dot Average Accuracy: 0.1743
[6] Validation Accuracy - Cross-to-Dot: 0.1619
[7] Cross-to-Dot Average Accuracy: 0.1743


[I 2024-06-26 17:33:29,957] Trial 0 finished with value: 0.1618652820587158 and parameters: {'lr': 0.0037986434408371333, 'weight_decay': 2.8692489739081363e-05, 'batch_size': 16}. Best is trial 0 with value: 0.1618652820587158.


[7] Validation Accuracy - Cross-to-Dot: 0.1619
Early stopping! Validation accuracy did not improve for the last 2 epochs.
[1] Cross-to-Dot Average Accuracy: 0.0020
[1] Validation Accuracy - Cross-to-Dot: 0.0000
[2] Cross-to-Dot Average Accuracy: 0.0000


[I 2024-06-26 17:37:49,211] Trial 1 finished with value: 0.0 and parameters: {'lr': 0.07676231496999499, 'weight_decay': 0.00017136367870661758, 'batch_size': 8}. Best is trial 0 with value: 0.1618652820587158.


...

[3] Validation Accuracy - Cross-to-Dot: 0.1736
Early stopping! Validation accuracy did not improve for the last 2 epochs.
[1] Cross-to-Dot Average Accuracy: 0.1538
[1] Validation Accuracy - Cross-to-Dot: 0.1404
[2] Cross-to-Dot Average Accuracy: 0.4033
[2] Validation Accuracy - Cross-to-Dot: 0.4321
[3] Cross-to-Dot Average Accuracy: 0.4362
[3] Validation Accuracy - Cross-to-Dot: 0.4226
[4] Cross-to-Dot Average Accuracy: 0.4374


[I 2024-06-26 17:57:29,289] Trial 4 finished with value: 0.4320603671826814 and parameters: {'lr': 0.001157028737328991, 'weight_decay': 0.05528001778458196, 'batch_size': 8}. Best is trial 4 with value: 0.4320603671826814.


[4] Validation Accuracy - Cross-to-Dot: 0.4239
Early stopping! Validation accuracy did not improve for the last 2 epochs.


[W 2024-06-26 17:57:30,740] Trial 5 failed with parameters: {'lr': 0.00010607834305447832, 'weight_decay': 0.00032910329842873585, 'batch_size': 32} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 1024.00 MiB. GPU ').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-17-e3a2d4e3b2ad>", line 37, in objective
    output_cross_to_dot = model(cross[0].float().to(device))
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1532, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1541, in _call_impl
    return forward_call(*args, **kwargs)
  File "<ipython-input-16-a0f6bc1a620c>", line 131, in forward
    out7 = self.conv7(out6 + out_f)
torch.cuda.OutOfMemoryError: CUDA out of memory. Tri

OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 

In [ ]:
# model = CNNP().to(device)
# print(model)

# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)

# epochs = 50
# batch_size = 4

# best_val_accuracy = 0.0
# patience = 2
# early_stop_counter = 0

# model.train()

# for epoch in range(epochs):
#     total_accuracy_cross_to_dot = 0
#     total_accuracy_dot_to_cross = 0

#     if epoch % 5 == 0:
#         # Save model parameters every 20 epochs
#         torch.save(model.state_dict(), '/content/drive/MyDrive/PROPOSED_SACNN/model_parameter/model_state.pthram' + str(epoch) + '.pth')

#     for i, (cross, dot) in enumerate(train_loader):
#         optimizer.zero_grad()

#         # Train for cross-to-dot complexity pairs
#         output_cross_to_dot = model(cross[0].float().to(device))
#         loss_cross_to_dot = criterion(output_cross_to_dot, dot[0].float().to(device))
#         loss_cross_to_dot.backward()
#         optimizer.step()

#         # Compute training accuracy for cross-to-dot complexity pairs
#         predictions_cross_to_dot = torch.round(output_cross_to_dot).squeeze().float()
#         correct_cross_to_dot = torch.eq(predictions_cross_to_dot, dot[0].float().to(device).squeeze()).sum().item()
#         accuracy_cross_to_dot_batch = correct_cross_to_dot / (batch_size * 512 * 512)
#         total_accuracy_cross_to_dot += accuracy_cross_to_dot_batch

#         # Similarly, train for dot-to-cross complexity pairs (modify as needed)

#     average_accuracy_cross_to_dot = total_accuracy_cross_to_dot / len(train_loader)
#     print(f'[{epoch + 1}] Cross-to-Dot Average Accuracy: {average_accuracy_cross_to_dot:.4f}')

#     # Validation step
#     model.eval()
#     with torch.no_grad():
#         total_val_accuracy_cross_to_dot = 0

#         for val_cross_img, val_dot_img in val_loader:
#             val_output_cross_to_dot = model(val_cross_img[0].float().to(device))
#             val_loss_cross_to_dot = criterion(val_output_cross_to_dot, val_dot_img[0].float().to(device))

#             val_predictions_cross_to_dot = torch.round(val_output_cross_to_dot).squeeze().float()
#             val_correct_cross_to_dot = torch.eq(val_predictions_cross_to_dot, val_dot_img[0].float().to(device).squeeze()).sum().item()
#             val_accuracy_cross_to_dot = val_correct_cross_to_dot / (batch_size * 512 * 512)

#             total_val_accuracy_cross_to_dot += val_accuracy_cross_to_dot

#         average_val_accuracy_cross_to_dot = total_val_accuracy_cross_to_dot / len(val_loader)
#         print(f'[{epoch + 1}] Validation Accuracy - Cross-to-Dot: {average_val_accuracy_cross_to_dot:.4f}')

#         # Check for improvement in validation accuracy
#         if average_val_accuracy_cross_to_dot > best_val_accuracy:
#             best_val_accuracy = average_val_accuracy_cross_to_dot
#             early_stop_counter = 0
#         else:
#             early_stop_counter += 1

#         # Check for early stopping
#         if early_stop_counter >= patience:
#             print("Early stopping! Validation accuracy did not improve for the last {} epochs.".format(patience))
#             break

#     scheduler.step()

#saving the model Weights

In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/PROPOSED_ACNN/model_parameter/model_state.pthpred.pth')

[link text](https://)#New Mutli attention


In [ ]:
class SelfAttention(nn.Module):

  def __init__(self,inplanes):
    super(SelfAttention,self).__init__()
    self.inplanes = inplanes
    # theta transform
    self.theta = nn.Conv2d(self.inplanes,self.inplanes//2,1)
    # phi transform
    self.phi = nn.Sequential(nn.Conv2d(self.inplanes,self.inplanes//2,1),
                             # nn.MaxPool2d(2,2)
                             )
    # function g
    self.g_func = nn.Sequential(nn.Conv2d(self.inplanes,self.inplanes//2,1),
                                # nn.MaxPool2d(2,2)
                                )

    # w transform to match the channel
    self.w = nn.Conv2d(self.inplanes//2,self.inplanes,1)

  def forward(self,xs):

    theta = self.theta(xs)
    N,C,W,H = theta.size()
    theta = theta.view(N,C,H*W).transpose(2,1)
    # print(theta.shape)
    phi = self.phi(xs)
    phi = phi.view(N,C,-1)
    # compute attention
    attention = theta.bmm(phi)
    assert attention.size()==(N,H*W,H*W)
    attention = nn.functional.softmax(attention,dim=-1)
    # g transform
    g = self.g_func(xs)
    g = g.view(N,C,-1)
    # final response
    response = g.bmm(attention.transpose(2,1))
    response = response.view(N,C,W,H)
    # matching channel
    response = self.w(response)
    output = response + xs
    return output



In [ ]:
class MA(nn.Module):
    def __init__(self, inplanes):
        super(MA, self).__init__()
        self.inplanes = inplanes

        # Define attention modules for each convolutional layer
        self.attention2 = SelfAttention(inplanes)
        self.attention3 = SelfAttention(inplanes)
        self.attention4 = SelfAttention(inplanes)
        self.attention5 = SelfAttention(inplanes)
        self.attention6 = SelfAttention(inplanes)
        self.attention7 = SelfAttention(inplanes)
        self.attention8 = SelfAttention(inplanes)
        self.attention9 = SelfAttention(inplanes)
        self.attention10 = SelfAttention(inplanes)

    def forward(self, xs):
        # Apply attention to each convolutional layer
        out2 = self.attention2(xs)
        out3 = self.attention3(xs)
        out4 = self.attention4(xs)
        out5 = self.attention5(xs)
        out6 = self.attention6(xs)
        out7 = self.attention7(xs)
        out8 = self.attention8(xs)
        out9 = self.attention9(xs)
        out10 = self.attention10(xs)

        return out10

In [ ]:
class CNNP(nn.Module):
    def __init__(self):
        super(CNNP, self).__init__()
        channel = 32

        self.conv2 = nn.Sequential(
            nn.Conv2d(1, channel, 5, 1, padding=2),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, channel, 3, 1, padding=1),
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(1, channel, 7, 1, padding=3),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, channel, 3, 1, padding=1),
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, channel, 3, 1, padding=1),
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.conv6 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.conv7 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.conv8 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.conv9 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )

        self.conv10 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, 1, 3, 1, padding=1),
        )
        # Define other convolution layers

        self.multi_attention = MultiAttention(channel)
        self.pool = nn.AvgPool2d(2, 2)
        self.resize = transforms.Resize(512)

    def forward(self, images):
        out2 = self.conv2(images)
        out3 = self.conv3(images)
        pool2 = self.pool(self.pool(self.pool(out2)))
        pool3 = self.pool(self.pool(self.pool(out3)))

        attn2 = self.multi_attention(pool2)
        attn3 = self.multi_attention(pool3)

        attn2 = self.resize(attn2)
        attn3 = self.resize(attn3)

        out_f = out3 + attn3 + out2 + attn2
        out4 = self.conv4(out_f)
        out5 = self.conv5(out4 + out_f)
        out6 = self.conv6(out4 + out_f)
        out7 = self.conv7(out6 + out_f)
        out8 = self.conv8(out7 + out_f)
        out9 = self.conv9(out8 + out_f)
        out10 = self.conv10(out9 + out_f)

        return out10

NEW CNN ARCHITECTURE

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

In [ ]:
class MHA(nn.Module):
    def __init__(self, inplanes, num_heads=8):
        super(MHA, self).__init__()
        self.inplanes = inplanes
        self.num_heads = num_heads
        assert self.inplanes % self.num_heads == 0, "inplanes must be divisible by num_heads"

        self.head_dim = self.inplanes // self.num_heads

        # Define linear transformations for queries, keys, and values for each head
        self.query_heads = nn.ModuleList([nn.Conv2d(self.inplanes, self.head_dim, 1) for _ in range(self.num_heads)])
        self.key_heads = nn.ModuleList([nn.Conv2d(self.inplanes, self.head_dim, 1) for _ in range(self.num_heads)])
        self.value_heads = nn.ModuleList([nn.Conv2d(self.inplanes, self.head_dim, 1) for _ in range(self.num_heads)])

        # Final linear transformation
        self.w = nn.Conv2d(self.inplanes, self.inplanes, 1)

    def forward(self, xs):
        # Apply linear transformations for query, key, value
        queries = [self.query_heads[i](xs) for i in range(self.num_heads)]
        keys = [self.key_heads[i](xs) for i in range(self.num_heads)]
        values = [self.value_heads[i](xs) for i in range(self.num_heads)]

        # Prepare query, key, value tensors
        queries = [query.view(query.size(0), self.head_dim, -1) for query in queries]
        keys = [key.view(key.size(0), self.head_dim, -1) for key in keys]
        values = [value.view(value.size(0), self.head_dim, -1) for value in values]

        # Compute attention scores
        attention_scores = [torch.matmul(queries[i], keys[i].transpose(-2, -1)) / (self.head_dim ** 0.5) for i in range(self.num_heads)]
        attention_scores = [F.softmax(score, dim=-1) for score in attention_scores]

        # Apply attention to values
        head_outputs = [torch.matmul(attention_scores[i], values[i]) for i in range(self.num_heads)]
        head_outputs = [output.view(output.size(0), self.head_dim, xs.size(2), xs.size(3)) for output in head_outputs]

        # Concatenate head outputs along the channel dimension
        multi_head_output = torch.cat(head_outputs, dim=1)

        # Final linear transformation
        multi_head_output = self.w(multi_head_output)

        # Add residual connection
        output = xs + multi_head_output
        return output


class CNNP(nn.Module):
    def __init__(self, channel=32):
        super(CNNP, self).__init__()
        self.channel = channel

        # Define convolutional layers
        self.conv2 = nn.Sequential(
            nn.Conv2d(1, channel, 5, 1, padding=2),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, channel, 3, 1, padding=1),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(1, channel, 7, 1, padding=3),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, channel, 3, 1, padding=1),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(channel, channel, 3, 1, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(channel, channel, 3, 1, padding=1),
        )
        self.conv5 = nn.Conv2d(channel, 1, 3, 1, padding=1)

        # Multi-head attention
        self.attn = MultiHeadAttention(channel, num_heads=8)
        self.pool = nn.AvgPool2d(2, 2)
        self.resize = transforms.Resize(512)

    def forward(self, images):
        # CNN layers
        out2 = self.conv2(images)
        out3 = self.conv3(images)
        pool2 = self.pool(self.pool(self.pool(out2)))
        pool3 = self.pool(self.pool(self.pool(out3)))

        # Apply multi-head attention
        attn2 = self.attn(pool2)
        attn3 = self.attn(pool3)
        attn2 = self.resize(attn2)
        attn3 = self.resize(attn3)

        # Final feature fusion
        out_f = out3 + attn3 + out2 + attn2
        out4 = self.conv4(out_f)
        out5 = self.conv5(out4)

        return out5